In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 135004 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.14-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.14-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.14-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redir

In [2]:
pip install MiniSom

  Created wheel for MiniSom: filename=MiniSom-2.2.3-cp36-none-any.whl size=7550 sha256=25271ea20b0ed1be76f08a8fe9e8561556ac6f3fb529963138bd99b0f2391814
  Stored in directory: /root/.cache/pip/wheels/35/62/79/4d921062c847ee15a3bb3ac2ea984ed401c8b6b2944f08f697
Successfully built MiniSom


In [0]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from datetime import datetime

from minisom import MiniSom
import matplotlib.pyplot as plt

In [0]:
class Data:
    # Load Adult dataset and seperate to features(X) and target(y)
    def __init__(self, path='adult.csv'):
        df = shuffle(pd.read_csv(path, engine='python'))
        df = self.clean(df)

        self.y = df.pop('income')
        self.X = df

        # Label encode y
        self.y_encoder = LabelEncoder()
        self.y = self.y_encoder.fit_transform(self.y)

        # One Hot encode X
        self.X = pd.get_dummies(self.X)

        for name in self.X.columns:
            if self.X[name].dtype == 'object':
                self.X[name] = self.X[name].astype('category')

    def clean(self, df):
        return df.replace('?', np.nan).dropna().drop('fnlwgt', axis=1)

    def train_test_split(self):
        X_train, X_test, y_train, y_test = train_test_split(
            self.X, self.y, test_size=0.2)
        y_train = pd.Series(y_train, index=X_train.index)
        y_test = pd.Series(y_test, index=X_test.index)
        return (X_train, X_test, y_train, y_test)

#SOM

In [3]:
som_start=datetime.now()

data = Data()

quasi_identifiers = ['age', 'educational-num',
                         'capital-gain', 'capital-loss', 'hours-per-week']
features = data.X[quasi_identifiers]
target = data.y

#要調整
width=150
height=150
sigma=.9
lr=.2
epochs=1e5
#epochs = 100

verbose=True
log = 1000

som = MiniSom(width, height, features.shape[1], sigma, lr)
som.train_random(features.values, int(epochs), verbose=True)

out = []
for step, (X, y) in enumerate(zip(features.values, target)):
    new_X = som.winner(X)
    out.append((new_X, X, y))
    if(verbose == True and step % log == 0):
        print(f'*Creating SOM: [{step}/{features.shape[0]}]')
som_data = np.array(out)

new_data = []
new_X = []
X = []
y = []
for i in range(0, len(som_data[:,0])):
    new_X.append(np.asarray(som_data[:,0][i]))
    X.append(np.asarray(som_data[:,1][i]))
    y.append(np.asarray(som_data[:,2][i]))
new_data = (new_X, X, y)

print("Time required for SOM: " + str(datetime.now()-som_start))

new_data

 [ 100000 / 100000 ] 100% - 0:00:00 left 
 quantization error: 104.0591413662037
 topographic error: 0.1506788731148556
*Creating SOM: [0/45222]
*Creating SOM: [1000/45222]
*Creating SOM: [2000/45222]
*Creating SOM: [3000/45222]
*Creating SOM: [4000/45222]
*Creating SOM: [5000/45222]
*Creating SOM: [6000/45222]
*Creating SOM: [7000/45222]
*Creating SOM: [8000/45222]
*Creating SOM: [9000/45222]
*Creating SOM: [10000/45222]
*Creating SOM: [11000/45222]
*Creating SOM: [12000/45222]
*Creating SOM: [13000/45222]
*Creating SOM: [14000/45222]
*Creating SOM: [15000/45222]
*Creating SOM: [16000/45222]
*Creating SOM: [17000/45222]
*Creating SOM: [18000/45222]
*Creating SOM: [19000/45222]
*Creating SOM: [20000/45222]
*Creating SOM: [21000/45222]
*Creating SOM: [22000/45222]
*Creating SOM: [23000/45222]
*Creating SOM: [24000/45222]
*Creating SOM: [25000/45222]
*Creating SOM: [26000/45222]
*Creating SOM: [27000/45222]
*Creating SOM: [28000/45222]
*Creating SOM: [29000/45222]
*Creating SOM: [30000/4

([array([43, 20]),
  array([42,  9]),
  array([38, 16]),
  array([33,  3]),
  array([20,  6]),
  array([30, 10]),
  array([38,  1]),
  array([42,  3]),
  array([19, 76]),
  array([54, 23]),
  array([33,  7]),
  array([34, 16]),
  array([26,  4]),
  array([54, 13]),
  array([37, 24]),
  array([41, 12]),
  array([21,  5]),
  array([32,  2]),
  array([18,  9]),
  array([33, 23]),
  array([17,  1]),
  array([47,  2]),
  array([19, 90]),
  array([24, 86]),
  array([32, 22]),
  array([43,  8]),
  array([31, 12]),
  array([45, 30]),
  array([47,  9]),
  array([22,  8]),
  array([40, 20]),
  array([53, 15]),
  array([28, 31]),
  array([33,  6]),
  array([38, 14]),
  array([35, 16]),
  array([24, 86]),
  array([40, 22]),
  array([23, 80]),
  array([43, 14]),
  array([33, 25]),
  array([23, 90]),
  array([19, 90]),
  array([35,  6]),
  array([41, 27]),
  array([36,  4]),
  array([36, 27]),
  array([13, 96]),
  array([18, 14]),
  array([52, 14]),
  array([41,  4]),
  array([49,  1]),
  array([23,

In [0]:
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from collections import Counter, defaultdict

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import Sequential
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score

In [0]:
class TrainingModel:
    def __init__(self, input_shape):
        self.model = Sequential()
        self.model.add(Dense(64, activation='relu', input_shape=input_shape))
        self.model.add(Dropout(0.3))
        self.model.add(Dense(128, activation='relu'))
        self.model.add(Dropout(0.3))
        self.model.add(Dense(128, activation='relu'))
        self.model.add(Dense(1, activation='sigmoid'))
        self.model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

    def fit(self, data, label):
        self.model.fit(data, label, epochs=1, batch_size=128, verbose=0)

    def predict(self, data):
        return self.model.predict_classes(data)
    
    def evaluate(self, X_test, y_test, print_report=True):
        y_predicted = self.predict(X_test)
        y_predicted_probs = self.model.predict_proba(X_test)
        if print_report:
            self.print_report(y_test, y_predicted, y_predicted_probs)
        else:
            accuracy = accuracy_score(y_test, y_predicted)
            report = classification_report(y_test, y_predicted, output_dict=True)
            auc_score = roc_auc_score(y_test, y_predicted_probs)
            matrix = confusion_matrix(y_test, y_predicted)

            return {
                'accuracy': accuracy,
                'auc_score': auc_score,
                **report['weighted avg'],
            }

    def print_report(self, test, predicted, predicted_probs):
        accuracy = accuracy_score(test, predicted)
        report = classification_report(test, predicted)
        matrix = confusion_matrix(test, predicted)

        print('Accuracy score: {:.5f}'.format(accuracy))
        print('-' * 20)
        print('Confusion Matrix:')
        print(matrix)
        print('-' * 20)
        print(report)
        print('-' * 20)
        print('AUC score: {:.5f}'.format(roc_auc_score(test, predicted_probs)))

#K-Anonymity

##k = 50

###K means

In [6]:
#將SOM資料集丟入Kmeans
#Kmeans分群數依照k值(k-anonymity的k)決定，從K = x/k開始遞減直到每一群至少有k筆資料
#計算跑的時間
#sizes = [5, 10, 15, 20, 30, 50]
k = 50
K = int(len(new_data[0])//(k*2))
D = 250//k #Decline
cluster = True
#c = Counter()

#for k in sizes:
#    print("K-Anonymity: k=" + str(k))
#    K = int(len(new_data[0])/(k*2))
#    cluster = True
kmeans_start = datetime.now()
while cluster:
    clf = MiniBatchKMeans(n_clusters=K)
    clf.fit(new_data[0])
    c = Counter(clf.labels_)
    print("K=" + str(K))
    for i in range(0,K):
        print("cluster " + str(i) + " has " + str(c[i]) + " data points")
        if c[i]<k:
            break
        else:
            if i == K-1:
                cluster = False
            else:
                pass
    K = K-D
K = K+D
print("The Resulting number of clusters: " + str(K))
print("Time required for Kmeans: " + str(datetime.now()-kmeans_start))

/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=452. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=452. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=452. Setting it to 3*k
  init_size=init_size)


K=452
cluster 0 has 120 data points
cluster 1 has 312 data points
cluster 2 has 218 data points
cluster 3 has 81 data points
cluster 4 has 51 data points
cluster 5 has 131 data points
cluster 6 has 115 data points
cluster 7 has 196 data points
cluster 8 has 152 data points
cluster 9 has 116 data points
cluster 10 has 156 data points
cluster 11 has 85 data points
cluster 12 has 104 data points
cluster 13 has 98 data points
cluster 14 has 37 data points


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=447. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=447. Setting it to 3*k
  init_size=init_size)


K=447
cluster 0 has 96 data points
cluster 1 has 21 data points


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=447. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=442. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=442. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=442. Setting it to 3*k
  init_size=init_size)


...

K=312
cluster 0 has 299 data points
cluster 1 has 78 data points
cluster 2 has 185 data points
cluster 3 has 237 data points
cluster 4 has 123 data points
cluster 5 has 158 data points
cluster 6 has 133 data points
cluster 7 has 171 data points
cluster 8 has 177 data points
cluster 9 has 235 data points
cluster 10 has 221 data points
cluster 11 has 55 data points
cluster 12 has 181 data points
cluster 13 has 289 data points
cluster 14 has 104 data points
cluster 15 has 158 data points
cluster 16 has 141 data points
cluster 17 has 205 data points
cluster 18 has 157 data points
cluster 19 has 200 data points
cluster 20 has 169 data points
cluster 21 has 113 data points
cluster 22 has 139 data points
cluster 23 has 121 data points
cluster 24 has 167 data points
cluster 25 has 211 data points
cluster 26 has 307 data points
cluster 27 has 115 data points
cluster 28 has 195 data points
cluster 29 has 136 data points
cluster 30 has 102 data points
cluster 31 has 191 data points
cluster 32 has

/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=307. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=307. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=307. Setting it to 3*k
  init_size=init_size)


K=307
cluster 0 has 49 data points


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=302. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=302. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=302. Setting it to 3*k
  init_size=init_size)


K=302
cluster 0 has 129 data points
cluster 1 has 287 data points
cluster 2 has 230 data points
cluster 3 has 165 data points
cluster 4 has 261 data points
cluster 5 has 411 data points
cluster 6 has 109 data points
cluster 7 has 39 data points
K=297
cluster 0 has 200 data points
cluster 1 has 77 data points
cluster 2 has 109 data points
cluster 3 has 351 data points
cluster 4 has 166 data points
cluster 5 has 249 data points
cluster 6 has 162 data points
cluster 7 has 28 data points
K=292
cluster 0 has 249 data points
cluster 1 has 84 data points
cluster 2 has 127 data points
cluster 3 has 112 data points
cluster 4 has 127 data points
cluster 5 has 219 data points
cluster 6 has 116 data points
cluster 7 has 148 data points
cluster 8 has 158 data points
cluster 9 has 187 data points
cluster 10 has 454 data points
cluster 11 has 151 data points
cluster 12 has 58 data points
cluster 13 has 250 data points
cluster 14 has 212 data points
cluster 15 has 224 data points
cluster 16 has 157 da

###Prepare perturbed dataset

In [7]:
#將分完群的資料還原回原始資料的feature
#準備丟入神經網路

perturb_start=datetime.now()

data = pd.concat([pd.DataFrame(new_data[1],columns=['age', 'educational-num','capital-gain', 'capital-loss', 'hours-per-week']), 
                  pd.DataFrame(new_data[2],columns=['target']), pd.DataFrame(clf.labels_,columns=['cluster'])], axis = 1)
columns = ['age', 'educational-num','capital-gain', 'capital-loss', 'hours-per-week','target','cluster']
index = range(0,len(data))
data_perturbed = pd.DataFrame(index = index, columns=columns)

for i in range(0, len(data)):
    for j in range(0,K+1):
        if data['cluster'][i] == j:
            data_perturbed[columns[0]][i] = float(data.groupby(by='cluster').mean()[columns[0]][j])
            data_perturbed[columns[1]][i] = float(data.groupby(by='cluster').mean()[columns[1]][j])
            data_perturbed[columns[2]][i] = float(data.groupby(by='cluster').mean()[columns[2]][j])
            data_perturbed[columns[3]][i] = float(data.groupby(by='cluster').mean()[columns[3]][j])
            data_perturbed[columns[4]][i] = float(data.groupby(by='cluster').mean()[columns[4]][j])
            data_perturbed['target'][i] = data['target'][i]
            data_perturbed['cluster'][i] = data['cluster'][i]

data_sorted = data_perturbed.sort_values(by=['cluster'])
data_ready50 = data_sorted.drop('cluster', axis=1)
print(data_ready50)

print("Time required for perturbation: " + str(datetime.now()-perturb_start))

           age educational-num capital-gain capital-loss hours-per-week target
11557  33.7625         9.24167            0            0        49.7833      0
11440  33.7625         9.24167            0            0        49.7833      0
11373  33.7625         9.24167            0            0        49.7833      1
11367  33.7625         9.24167            0            0        49.7833      1
39706  33.7625         9.24167            0            0        49.7833      0
...        ...             ...          ...          ...            ...    ...
8953      36.8         9.87273      3119.31            0           41.6      0
27689     36.8         9.87273      3119.31            0           41.6      0
33064     36.8         9.87273      3119.31            0           41.6      1
5342      36.8         9.87273      3119.31            0           41.6      0
4371      36.8         9.87273      3119.31            0           41.6      0

[45222 rows x 6 columns]
Time required for perturba

###Model

In [8]:
X_train, X_test, y_train, y_test = train_test_split(data_ready50.iloc[:,0:5], np.array(data_ready50.iloc[:,5:], dtype=int), test_size=0.2)
model50 = TrainingModel((5,))
model50.fit(X_train, y_train)
model50.evaluate(X_test, y_test, print_report=True)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Accuracy score: 0.77402
--------------------
Confusion Matrix:
[[6520  264]
 [1780  481]]
--------------------
              precision    recall  f1-score   support

           0       0.79      0.96      0.86      6784
           1       0.65      0.21      0.32      2261

    accuracy                           0.77      9045
   macro avg       0.72      0.59      0.59      9045
weighted avg       0.75      0.77      0.73      9045

--------------------
AUC score: 0.57046


##k=30

###K means

In [11]:
#將SOM資料集丟入Kmeans
#Kmeans分群數依照k值(k-anonymity的k)決定，從K = x/k開始遞減直到每一群至少有k筆資料
#計算跑的時間
#sizes = [5, 10, 15, 20, 30, 50]
k = 30
K = int(len(new_data[0])//(k*2))
D = 250//k #Decline
cluster = True
#c = Counter()

#for k in sizes:
#    print("K-Anonymity: k=" + str(k))
#    K = int(len(new_data[0])/(k*2))
#    cluster = True
kmeans_start = datetime.now()
while cluster:
    clf = MiniBatchKMeans(n_clusters=K)
    clf.fit(new_data[0])
    c = Counter(clf.labels_)
    print("K=" + str(K))
    for i in range(0,K):
        print("cluster " + str(i) + " has " + str(c[i]) + " data points")
        if c[i]<k:
            break
        else:
            if i == K-1:
                cluster = False
            else:
                pass
    K = K-D
K = K+D
print("The Resulting number of clusters: " + str(K))
print("Time required for Kmeans: " + str(datetime.now()-kmeans_start))

/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=753. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=753. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=753. Setting it to 3*k
  init_size=init_size)


K=753
cluster 0 has 203 data points
cluster 1 has 37 data points
cluster 2 has 82 data points
cluster 3 has 170 data points
cluster 4 has 78 data points
cluster 5 has 46 data points
cluster 6 has 105 data points
cluster 7 has 211 data points
cluster 8 has 0 data points


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=745. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=745. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=745. Setting it to 3*k
  init_size=init_size)


K=745
cluster 0 has 27 data points


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=737. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=737. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=737. Setting it to 3*k
  init_size=init_size)


...

K=321
cluster 0 has 127 data points
cluster 1 has 287 data points
cluster 2 has 210 data points
cluster 3 has 205 data points
cluster 4 has 534 data points
cluster 5 has 339 data points
cluster 6 has 122 data points
cluster 7 has 128 data points
cluster 8 has 138 data points
cluster 9 has 113 data points
cluster 10 has 130 data points
cluster 11 has 161 data points
cluster 12 has 51 data points
cluster 13 has 58 data points
cluster 14 has 128 data points
cluster 15 has 104 data points
cluster 16 has 162 data points
cluster 17 has 111 data points
cluster 18 has 218 data points
cluster 19 has 84 data points
cluster 20 has 115 data points
cluster 21 has 177 data points
cluster 22 has 289 data points
cluster 23 has 62 data points
cluster 24 has 145 data points
cluster 25 has 113 data points
cluster 26 has 423 data points
cluster 27 has 290 data points
cluster 28 has 92 data points
cluster 29 has 94 data points
cluster 30 has 31 data points
cluster 31 has 161 data points
cluster 32 has 213 

/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=313. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=313. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=313. Setting it to 3*k
  init_size=init_size)


K=313
cluster 0 has 95 data points
cluster 1 has 140 data points
cluster 2 has 239 data points
cluster 3 has 146 data points
cluster 4 has 172 data points
cluster 5 has 144 data points
cluster 6 has 221 data points
cluster 7 has 79 data points
cluster 8 has 181 data points
cluster 9 has 167 data points
cluster 10 has 201 data points
cluster 11 has 141 data points
cluster 12 has 175 data points
cluster 13 has 63 data points
cluster 14 has 227 data points
cluster 15 has 236 data points
cluster 16 has 165 data points
cluster 17 has 57 data points
cluster 18 has 175 data points
cluster 19 has 139 data points
cluster 20 has 145 data points
cluster 21 has 203 data points
cluster 22 has 119 data points
cluster 23 has 159 data points
cluster 24 has 92 data points
cluster 25 has 0 data points


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=305. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=305. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=305. Setting it to 3*k
  init_size=init_size)


K=305
cluster 0 has 195 data points
cluster 1 has 90 data points
cluster 2 has 312 data points
cluster 3 has 327 data points
cluster 4 has 162 data points
cluster 5 has 158 data points
cluster 6 has 468 data points
cluster 7 has 90 data points
cluster 8 has 305 data points
cluster 9 has 159 data points
cluster 10 has 290 data points
cluster 11 has 123 data points
cluster 12 has 67 data points
cluster 13 has 200 data points
cluster 14 has 69 data points
cluster 15 has 149 data points
cluster 16 has 270 data points
cluster 17 has 217 data points
cluster 18 has 117 data points
cluster 19 has 323 data points
cluster 20 has 297 data points
cluster 21 has 126 data points
cluster 22 has 96 data points
cluster 23 has 80 data points
cluster 24 has 257 data points
cluster 25 has 204 data points
cluster 26 has 102 data points
cluster 27 has 304 data points
cluster 28 has 112 data points
cluster 29 has 34 data points
cluster 30 has 25 data points
K=297
cluster 0 has 211 data points
cluster 1 has 8

###Prepare perturbed dataset

In [12]:
#將分完群的資料還原回原始資料的feature
#準備丟入神經網路

perturb_start=datetime.now()

data = pd.concat([pd.DataFrame(new_data[1],columns=['age', 'educational-num','capital-gain', 'capital-loss', 'hours-per-week']), 
                  pd.DataFrame(new_data[2],columns=['target']), pd.DataFrame(clf.labels_,columns=['cluster'])], axis = 1)
columns = ['age', 'educational-num','capital-gain', 'capital-loss', 'hours-per-week','target','cluster']
index = range(0,len(data))
data_perturbed = pd.DataFrame(index = index, columns=columns)

for i in range(0, len(data)):
    for j in range(0,K+1):
        if data['cluster'][i] == j:
            data_perturbed[columns[0]][i] = float(data.groupby(by='cluster').mean()[columns[0]][j])
            data_perturbed[columns[1]][i] = float(data.groupby(by='cluster').mean()[columns[1]][j])
            data_perturbed[columns[2]][i] = float(data.groupby(by='cluster').mean()[columns[2]][j])
            data_perturbed[columns[3]][i] = float(data.groupby(by='cluster').mean()[columns[3]][j])
            data_perturbed[columns[4]][i] = float(data.groupby(by='cluster').mean()[columns[4]][j])
            data_perturbed['target'][i] = data['target'][i]
            data_perturbed['cluster'][i] = data['cluster'][i]

data_sorted = data_perturbed.sort_values(by=['cluster'])
data_ready30 = data_sorted.drop('cluster', axis=1)
print(data_ready30)

print("Time required for perturbation: " + str(datetime.now()-perturb_start))

           age educational-num capital-gain capital-loss hours-per-week target
32066  39.0994         11.9689            0            0        39.8137      0
3838   39.0994         11.9689            0            0        39.8137      0
13107  39.0994         11.9689            0            0        39.8137      0
13108  39.0994         11.9689            0            0        39.8137      0
14824  39.0994         11.9689            0            0        39.8137      0
...        ...             ...          ...          ...            ...    ...
25873  44.5155         11.3814            0            0        39.8969      0
17566  44.5155         11.3814            0            0        39.8969      1
35422  44.5155         11.3814            0            0        39.8969      0
15103  44.5155         11.3814            0            0        39.8969      0
34270  44.5155         11.3814            0            0        39.8969      1

[45222 rows x 6 columns]
Time required for perturba

###Model

In [13]:
X_train, X_test, y_train, y_test = train_test_split(data_ready30.iloc[:,0:5], np.array(data_ready30.iloc[:,5:], dtype=int), test_size=0.2)
model30 = TrainingModel((5,))
model30.fit(X_train, y_train)
model30.evaluate(X_test, y_test, print_report=True)

Accuracy score: 0.77557
--------------------
Confusion Matrix:
[[6299  505]
 [1525  716]]
--------------------
              precision    recall  f1-score   support

           0       0.81      0.93      0.86      6804
           1       0.59      0.32      0.41      2241

    accuracy                           0.78      9045
   macro avg       0.70      0.62      0.64      9045
weighted avg       0.75      0.78      0.75      9045

--------------------
AUC score: 0.59394


##k=20

###K Means

In [14]:
#將SOM資料集丟入Kmeans
#Kmeans分群數依照k值(k-anonymity的k)決定，從K = x/k開始遞減直到每一群至少有k筆資料
#計算跑的時間
#sizes = [5, 10, 15, 20, 30, 50]
k = 20
K = int(len(new_data[0])//(k*2))
D = 250//k #Decline
cluster = True
#c = Counter()

#for k in sizes:
#    print("K-Anonymity: k=" + str(k))
#    K = int(len(new_data[0])/(k*2))
#    cluster = True
kmeans_start = datetime.now()
while cluster:
    clf = MiniBatchKMeans(n_clusters=K)
    clf.fit(new_data[0])
    c = Counter(clf.labels_)
    print("K=" + str(K))
    for i in range(0,K):
        print("cluster " + str(i) + " has " + str(c[i]) + " data points")
        if c[i]<k:
            break
        else:
            if i == K-1:
                cluster = False
            else:
                pass
    K = K-D
K = K+D
print("The Resulting number of clusters: " + str(K))
print("Time required for Kmeans: " + str(datetime.now()-kmeans_start))

/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=1130. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=1130. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=1130. Setting it to 3*k
  init_size=init_size)


K=1130
cluster 0 has 63 data points
cluster 1 has 135 data points
cluster 2 has 215 data points
cluster 3 has 82 data points
cluster 4 has 54 data points
cluster 5 has 57 data points
cluster 6 has 31 data points
cluster 7 has 115 data points
cluster 8 has 55 data points
cluster 9 has 131 data points
cluster 10 has 68 data points
cluster 11 has 3 data points


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=1118. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=1118. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=1118. Setting it to 3*k
  init_size=init_size)


K=1118
cluster 0 has 76 data points
cluster 1 has 76 data points
cluster 2 has 151 data points
cluster 3 has 47 data points
cluster 4 has 58 data points
cluster 5 has 103 data points
cluster 6 has 212 data points
cluster 7 has 94 data points
cluster 8 has 211 data points
cluster 9 has 105 data points
cluster 10 has 109 data points
cluster 11 has 111 data points
cluster 12 has 19 data points


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=1106. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=1106. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=1106. Setting it to 3*k
  init_size=init_size)


...

K=326
cluster 0 has 116 data points
cluster 1 has 512 data points
cluster 2 has 154 data points
cluster 3 has 86 data points
cluster 4 has 25 data points
cluster 5 has 277 data points
cluster 6 has 194 data points
cluster 7 has 69 data points
cluster 8 has 346 data points
cluster 9 has 211 data points
cluster 10 has 78 data points
cluster 11 has 106 data points
cluster 12 has 122 data points
cluster 13 has 146 data points
cluster 14 has 156 data points
cluster 15 has 113 data points
cluster 16 has 174 data points
cluster 17 has 166 data points
cluster 18 has 152 data points
cluster 19 has 179 data points
cluster 20 has 84 data points
cluster 21 has 140 data points
cluster 22 has 135 data points
cluster 23 has 95 data points
cluster 24 has 70 data points
cluster 25 has 292 data points
cluster 26 has 112 data points
cluster 27 has 154 data points
cluster 28 has 107 data points
cluster 29 has 309 data points
cluster 30 has 90 data points
cluster 31 has 91 data points
cluster 32 has 121 da

/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=314. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=314. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=314. Setting it to 3*k
  init_size=init_size)


K=314
cluster 0 has 81 data points
cluster 1 has 75 data points
cluster 2 has 122 data points
cluster 3 has 173 data points
cluster 4 has 270 data points
cluster 5 has 132 data points
cluster 6 has 251 data points
cluster 7 has 95 data points
cluster 8 has 206 data points
cluster 9 has 141 data points
cluster 10 has 300 data points
cluster 11 has 230 data points
cluster 12 has 90 data points
cluster 13 has 136 data points
cluster 14 has 77 data points
cluster 15 has 214 data points
cluster 16 has 70 data points
cluster 17 has 49 data points
cluster 18 has 91 data points
cluster 19 has 264 data points
cluster 20 has 239 data points
cluster 21 has 148 data points
cluster 22 has 103 data points
cluster 23 has 116 data points
cluster 24 has 253 data points
cluster 25 has 47 data points
cluster 26 has 326 data points
cluster 27 has 161 data points
cluster 28 has 117 data points
cluster 29 has 130 data points
cluster 30 has 25 data points
cluster 31 has 106 data points
cluster 32 has 341 dat

/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=302. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=302. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=302. Setting it to 3*k
  init_size=init_size)


K=302
cluster 0 has 127 data points
cluster 1 has 297 data points
cluster 2 has 178 data points
cluster 3 has 238 data points
cluster 4 has 170 data points
cluster 5 has 81 data points
cluster 6 has 208 data points
cluster 7 has 195 data points
cluster 8 has 14 data points
K=290
cluster 0 has 201 data points
cluster 1 has 135 data points
cluster 2 has 214 data points
cluster 3 has 178 data points
cluster 4 has 129 data points
cluster 5 has 149 data points
cluster 6 has 126 data points
cluster 7 has 312 data points
cluster 8 has 149 data points
cluster 9 has 160 data points
cluster 10 has 217 data points
cluster 11 has 314 data points
cluster 12 has 270 data points
cluster 13 has 239 data points
cluster 14 has 111 data points
cluster 15 has 135 data points
cluster 16 has 287 data points
cluster 17 has 210 data points
cluster 18 has 257 data points
cluster 19 has 164 data points
cluster 20 has 154 data points
cluster 21 has 551 data points
cluster 22 has 41 data points
cluster 23 has 90 

###Prepare perturbed dataset

In [15]:
#將分完群的資料還原回原始資料的feature
#準備丟入神經網路

perturb_start=datetime.now()

data = pd.concat([pd.DataFrame(new_data[1],columns=['age', 'educational-num','capital-gain', 'capital-loss', 'hours-per-week']), 
                  pd.DataFrame(new_data[2],columns=['target']), pd.DataFrame(clf.labels_,columns=['cluster'])], axis = 1)
columns = ['age', 'educational-num','capital-gain', 'capital-loss', 'hours-per-week','target','cluster']
index = range(0,len(data))
data_perturbed = pd.DataFrame(index = index, columns=columns)

for i in range(0, len(data)):
    for j in range(0,K+1):
        if data['cluster'][i] == j:
            data_perturbed[columns[0]][i] = float(data.groupby(by='cluster').mean()[columns[0]][j])
            data_perturbed[columns[1]][i] = float(data.groupby(by='cluster').mean()[columns[1]][j])
            data_perturbed[columns[2]][i] = float(data.groupby(by='cluster').mean()[columns[2]][j])
            data_perturbed[columns[3]][i] = float(data.groupby(by='cluster').mean()[columns[3]][j])
            data_perturbed[columns[4]][i] = float(data.groupby(by='cluster').mean()[columns[4]][j])
            data_perturbed['target'][i] = data['target'][i]
            data_perturbed['cluster'][i] = data['cluster'][i]

data_sorted = data_perturbed.sort_values(by=['cluster'])
data_ready20 = data_sorted.drop('cluster', axis=1)
print(data_ready20)

print("Time required for perturbation: " + str(datetime.now()-perturb_start))

           age educational-num capital-gain capital-loss hours-per-week target
34135  47.4162         10.1878            0            0        39.9848      0
23324  47.4162         10.1878            0            0        39.9848      0
6936   47.4162         10.1878            0            0        39.9848      0
8923   47.4162         10.1878            0            0        39.9848      0
43669  47.4162         10.1878            0            0        39.9848      0
...        ...             ...          ...          ...            ...    ...
26960  19.9615         7.82692            0      1.49038        49.7788      0
26941  19.9615         7.82692            0      1.49038        49.7788      0
12397  19.9615         7.82692            0      1.49038        49.7788      0
11104  19.9615         7.82692            0      1.49038        49.7788      0
10400  19.9615         7.82692            0      1.49038        49.7788      0

[45222 rows x 6 columns]
Time required for perturba

###Model

In [16]:
X_train, X_test, y_train, y_test = train_test_split(data_ready20.iloc[:,0:5], np.array(data_ready20.iloc[:,5:], dtype=int), test_size=0.2)
model20 = TrainingModel((5,))
model20.fit(X_train, y_train)
model20.evaluate(X_test, y_test, print_report=True)

Accuracy score: 0.77855
--------------------
Confusion Matrix:
[[6569  264]
 [1739  473]]
--------------------
              precision    recall  f1-score   support

           0       0.79      0.96      0.87      6833
           1       0.64      0.21      0.32      2212

    accuracy                           0.78      9045
   macro avg       0.72      0.59      0.59      9045
weighted avg       0.75      0.78      0.73      9045

--------------------
AUC score: 0.51380


##k=15

###K means

In [17]:
#將SOM資料集丟入Kmeans
#Kmeans分群數依照k值(k-anonymity的k)決定，從K = x/k開始遞減直到每一群至少有k筆資料
#計算跑的時間
#sizes = [5, 10, 15, 20, 30, 50]
k = 15
K = int(len(new_data[0])//(k*2))
D = 250//k #Decline
cluster = True
#c = Counter()

#for k in sizes:
#    print("K-Anonymity: k=" + str(k))
#    K = int(len(new_data[0])/(k*2))
#    cluster = True
kmeans_start = datetime.now()
while cluster:
    clf = MiniBatchKMeans(n_clusters=K)
    clf.fit(new_data[0])
    c = Counter(clf.labels_)
    print("K=" + str(K))
    for i in range(0,K):
        print("cluster " + str(i) + " has " + str(c[i]) + " data points")
        if c[i]<k:
            break
        else:
            if i == K-1:
                cluster = False
            else:
                pass
    K = K-D
K = K+D
print("The Resulting number of clusters: " + str(K))
print("Time required for Kmeans: " + str(datetime.now()-kmeans_start))

/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=1507. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=1507. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=1507. Setting it to 3*k
  init_size=init_size)


K=1507
cluster 0 has 36 data points
cluster 1 has 116 data points
cluster 2 has 31 data points
cluster 3 has 57 data points
cluster 4 has 151 data points
cluster 5 has 30 data points
cluster 6 has 25 data points
cluster 7 has 165 data points
cluster 8 has 10 data points


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=1491. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=1491. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=1491. Setting it to 3*k
  init_size=init_size)


K=1491
cluster 0 has 76 data points
cluster 1 has 161 data points
cluster 2 has 74 data points
cluster 3 has 109 data points
cluster 4 has 34 data points
cluster 5 has 18 data points
cluster 6 has 105 data points
cluster 7 has 105 data points
cluster 8 has 157 data points
cluster 9 has 108 data points
cluster 10 has 56 data points
cluster 11 has 60 data points
cluster 12 has 206 data points
cluster 13 has 55 data points
cluster 14 has 225 data points
cluster 15 has 13 data points


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=1475. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=1475. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=1475. Setting it to 3*k
  init_size=init_size)


...

K=355
cluster 0 has 58 data points
cluster 1 has 137 data points
cluster 2 has 253 data points
cluster 3 has 135 data points
cluster 4 has 206 data points
cluster 5 has 177 data points
cluster 6 has 233 data points
cluster 7 has 75 data points
cluster 8 has 7 data points


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=339. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=339. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=339. Setting it to 3*k
  init_size=init_size)


K=339
cluster 0 has 224 data points
cluster 1 has 257 data points
cluster 2 has 50 data points
cluster 3 has 186 data points
cluster 4 has 177 data points
cluster 5 has 117 data points
cluster 6 has 155 data points
cluster 7 has 212 data points
cluster 8 has 296 data points
cluster 9 has 58 data points
cluster 10 has 42 data points
cluster 11 has 221 data points
cluster 12 has 145 data points
cluster 13 has 47 data points
cluster 14 has 132 data points
cluster 15 has 185 data points
cluster 16 has 113 data points
cluster 17 has 66 data points
cluster 18 has 116 data points
cluster 19 has 56 data points
cluster 20 has 187 data points
cluster 21 has 174 data points
cluster 22 has 133 data points
cluster 23 has 151 data points
cluster 24 has 138 data points
cluster 25 has 149 data points
cluster 26 has 135 data points
cluster 27 has 89 data points
cluster 28 has 171 data points
cluster 29 has 167 data points
cluster 30 has 89 data points
cluster 31 has 173 data points
cluster 32 has 39 da

/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=323. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=323. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=323. Setting it to 3*k
  init_size=init_size)


K=323
cluster 0 has 218 data points
cluster 1 has 84 data points
cluster 2 has 152 data points
cluster 3 has 298 data points
cluster 4 has 130 data points
cluster 5 has 68 data points
cluster 6 has 229 data points
cluster 7 has 166 data points
cluster 8 has 145 data points
cluster 9 has 248 data points
cluster 10 has 88 data points
cluster 11 has 108 data points
cluster 12 has 115 data points
cluster 13 has 122 data points
cluster 14 has 207 data points
cluster 15 has 211 data points
cluster 16 has 128 data points
cluster 17 has 229 data points
cluster 18 has 161 data points
cluster 19 has 189 data points
cluster 20 has 241 data points
cluster 21 has 167 data points
cluster 22 has 353 data points
cluster 23 has 167 data points
cluster 24 has 163 data points
cluster 25 has 90 data points
cluster 26 has 159 data points
cluster 27 has 143 data points
cluster 28 has 104 data points
cluster 29 has 48 data points
cluster 30 has 253 data points
cluster 31 has 156 data points
cluster 32 has 17

### Prepare perturbed dataset

In [18]:
#將分完群的資料還原回原始資料的feature
#準備丟入神經網路

perturb_start=datetime.now()

data = pd.concat([pd.DataFrame(new_data[1],columns=['age', 'educational-num','capital-gain', 'capital-loss', 'hours-per-week']), 
                  pd.DataFrame(new_data[2],columns=['target']), pd.DataFrame(clf.labels_,columns=['cluster'])], axis = 1)
columns = ['age', 'educational-num','capital-gain', 'capital-loss', 'hours-per-week','target','cluster']
index = range(0,len(data))
data_perturbed = pd.DataFrame(index = index, columns=columns)

for i in range(0, len(data)):
    for j in range(0,K+1):
        if data['cluster'][i] == j:
            data_perturbed[columns[0]][i] = float(data.groupby(by='cluster').mean()[columns[0]][j])
            data_perturbed[columns[1]][i] = float(data.groupby(by='cluster').mean()[columns[1]][j])
            data_perturbed[columns[2]][i] = float(data.groupby(by='cluster').mean()[columns[2]][j])
            data_perturbed[columns[3]][i] = float(data.groupby(by='cluster').mean()[columns[3]][j])
            data_perturbed[columns[4]][i] = float(data.groupby(by='cluster').mean()[columns[4]][j])
            data_perturbed['target'][i] = data['target'][i]
            data_perturbed['cluster'][i] = data['cluster'][i]

data_sorted = data_perturbed.sort_values(by=['cluster'])
data_ready15 = data_sorted.drop('cluster', axis=1)
print(data_ready15)

print("Time required for perturbation: " + str(datetime.now()-perturb_start))

           age educational-num capital-gain capital-loss hours-per-week target
7572   38.5872         10.5275            0            0        61.1606      0
25867  38.5872         10.5275            0            0        61.1606      0
1985   38.5872         10.5275            0            0        61.1606      1
7196   38.5872         10.5275            0            0        61.1606      1
18574  38.5872         10.5275            0            0        61.1606      0
...        ...             ...          ...          ...            ...    ...
43074  63.1395         10.2558            0            0        44.7209      0
13874  63.1395         10.2558            0            0        44.7209      0
5046   63.1395         10.2558            0            0        44.7209      1
23336  63.1395         10.2558            0            0        44.7209      1
14822  63.1395         10.2558            0            0        44.7209      0

[45222 rows x 6 columns]
Time required for perturba

###Model

In [19]:
X_train, X_test, y_train, y_test = train_test_split(data_ready15.iloc[:,0:5], np.array(data_ready15.iloc[:,5:], dtype=int), test_size=0.2)
model15 = TrainingModel((5,))
model15.fit(X_train, y_train)
model15.evaluate(X_test, y_test, print_report=True)

Accuracy score: 0.77324
--------------------
Confusion Matrix:
[[6473  287]
 [1764  521]]
--------------------
              precision    recall  f1-score   support

           0       0.79      0.96      0.86      6760
           1       0.64      0.23      0.34      2285

    accuracy                           0.77      9045
   macro avg       0.72      0.59      0.60      9045
weighted avg       0.75      0.77      0.73      9045

--------------------
AUC score: 0.66150


##k=10

###K means

In [20]:
#將SOM資料集丟入Kmeans
#Kmeans分群數依照k值(k-anonymity的k)決定，從K = x/k開始遞減直到每一群至少有k筆資料
#計算跑的時間
#sizes = [5, 10, 15, 20, 30, 50]
k = 10
K = int(len(new_data[0])//(k*2))
D = 250//k #Decline
cluster = True
#c = Counter()

#for k in sizes:
#    print("K-Anonymity: k=" + str(k))
#    K = int(len(new_data[0])/(k*2))
#    cluster = True
kmeans_start = datetime.now()
while cluster:
    clf = MiniBatchKMeans(n_clusters=K)
    clf.fit(new_data[0])
    c = Counter(clf.labels_)
    print("K=" + str(K))
    for i in range(0,K):
        print("cluster " + str(i) + " has " + str(c[i]) + " data points")
        if c[i]<k:
            break
        else:
            if i == K-1:
                cluster = False
            else:
                pass
    K = K-D
K = K+D
print("The Resulting number of clusters: " + str(K))
print("Time required for Kmeans: " + str(datetime.now()-kmeans_start))

/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=2261. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=2261. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=2261. Setting it to 3*k
  init_size=init_size)


K=2261
cluster 0 has 28 data points
cluster 1 has 114 data points
cluster 2 has 56 data points
cluster 3 has 87 data points
cluster 4 has 207 data points
cluster 5 has 31 data points
cluster 6 has 202 data points
cluster 7 has 19 data points
cluster 8 has 81 data points
cluster 9 has 94 data points
cluster 10 has 79 data points
cluster 11 has 34 data points
cluster 12 has 82 data points
cluster 13 has 106 data points
cluster 14 has 47 data points
cluster 15 has 139 data points
cluster 16 has 123 data points
cluster 17 has 168 data points
cluster 18 has 23 data points
cluster 19 has 19 data points
cluster 20 has 28 data points
cluster 21 has 9 data points


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=2236. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=2236. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=2236. Setting it to 3*k
  init_size=init_size)


K=2236
cluster 0 has 84 data points
cluster 1 has 3 data points


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=2211. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=2211. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=2211. Setting it to 3*k
  init_size=init_size)


...

K=411
cluster 0 has 222 data points
cluster 1 has 24 data points
cluster 2 has 215 data points
cluster 3 has 71 data points
cluster 4 has 138 data points
cluster 5 has 163 data points
cluster 6 has 159 data points
cluster 7 has 163 data points
cluster 8 has 57 data points
cluster 9 has 95 data points
cluster 10 has 117 data points
cluster 11 has 78 data points
cluster 12 has 48 data points
cluster 13 has 208 data points
cluster 14 has 85 data points
cluster 15 has 76 data points
cluster 16 has 111 data points
cluster 17 has 52 data points
cluster 18 has 135 data points
cluster 19 has 133 data points
cluster 20 has 74 data points
cluster 21 has 202 data points
cluster 22 has 141 data points
cluster 23 has 18 data points
cluster 24 has 145 data points
cluster 25 has 96 data points
cluster 26 has 512 data points
cluster 27 has 109 data points
cluster 28 has 134 data points
cluster 29 has 88 data points
cluster 30 has 216 data points
cluster 31 has 74 data points
cluster 32 has 105 data po

/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=386. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=386. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=386. Setting it to 3*k
  init_size=init_size)


K=386
cluster 0 has 147 data points
cluster 1 has 84 data points
cluster 2 has 89 data points
cluster 3 has 292 data points
cluster 4 has 82 data points
cluster 5 has 77 data points
cluster 6 has 30 data points
cluster 7 has 127 data points
cluster 8 has 135 data points
cluster 9 has 24 data points
cluster 10 has 192 data points
cluster 11 has 113 data points
cluster 12 has 201 data points
cluster 13 has 117 data points
cluster 14 has 106 data points
cluster 15 has 115 data points
cluster 16 has 289 data points
cluster 17 has 52 data points
cluster 18 has 56 data points
cluster 19 has 182 data points
cluster 20 has 176 data points
cluster 21 has 169 data points
cluster 22 has 41 data points
cluster 23 has 191 data points
cluster 24 has 56 data points
cluster 25 has 148 data points
cluster 26 has 61 data points
cluster 27 has 26 data points
cluster 28 has 163 data points
cluster 29 has 115 data points
cluster 30 has 208 data points
cluster 31 has 113 data points
cluster 32 has 137 data 

/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=361. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=361. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=361. Setting it to 3*k
  init_size=init_size)


K=361
cluster 0 has 132 data points
cluster 1 has 579 data points
cluster 2 has 110 data points
cluster 3 has 83 data points
cluster 4 has 127 data points
cluster 5 has 145 data points
cluster 6 has 26 data points
cluster 7 has 319 data points
cluster 8 has 216 data points
cluster 9 has 154 data points
cluster 10 has 29 data points
cluster 11 has 29 data points
cluster 12 has 161 data points
cluster 13 has 239 data points
cluster 14 has 99 data points
cluster 15 has 65 data points
cluster 16 has 155 data points
cluster 17 has 160 data points
cluster 18 has 81 data points
cluster 19 has 289 data points
cluster 20 has 87 data points
cluster 21 has 180 data points
cluster 22 has 73 data points
cluster 23 has 52 data points
cluster 24 has 154 data points
cluster 25 has 281 data points
cluster 26 has 146 data points
cluster 27 has 399 data points
cluster 28 has 188 data points
cluster 29 has 123 data points
cluster 30 has 58 data points
cluster 31 has 162 data points
cluster 32 has 76 data 

###Prepare perturbed dataset

In [21]:
#將分完群的資料還原回原始資料的feature
#準備丟入神經網路

perturb_start=datetime.now()

data = pd.concat([pd.DataFrame(new_data[1],columns=['age', 'educational-num','capital-gain', 'capital-loss', 'hours-per-week']), 
                  pd.DataFrame(new_data[2],columns=['target']), pd.DataFrame(clf.labels_,columns=['cluster'])], axis = 1)
columns = ['age', 'educational-num','capital-gain', 'capital-loss', 'hours-per-week','target','cluster']
index = range(0,len(data))
data_perturbed = pd.DataFrame(index = index, columns=columns)

for i in range(0, len(data)):
    for j in range(0,K+1):
        if data['cluster'][i] == j:
            data_perturbed[columns[0]][i] = float(data.groupby(by='cluster').mean()[columns[0]][j])
            data_perturbed[columns[1]][i] = float(data.groupby(by='cluster').mean()[columns[1]][j])
            data_perturbed[columns[2]][i] = float(data.groupby(by='cluster').mean()[columns[2]][j])
            data_perturbed[columns[3]][i] = float(data.groupby(by='cluster').mean()[columns[3]][j])
            data_perturbed[columns[4]][i] = float(data.groupby(by='cluster').mean()[columns[4]][j])
            data_perturbed['target'][i] = data['target'][i]
            data_perturbed['cluster'][i] = data['cluster'][i]

data_sorted = data_perturbed.sort_values(by=['cluster'])
data_ready10 = data_sorted.drop('cluster', axis=1)
print(data_ready10)

print("Time required for perturbation: " + str(datetime.now()-perturb_start))

           age educational-num capital-gain capital-loss hours-per-week target
19208  63.6818         8.07576            0            0        50.4697      0
5834   63.6818         8.07576            0            0        50.4697      1
41525  63.6818         8.07576            0            0        50.4697      0
39416  63.6818         8.07576            0            0        50.4697      0
38533  63.6818         8.07576            0            0        50.4697      0
...        ...             ...          ...          ...            ...    ...
15563  54.9292         13.6814            0            0        41.6372      1
37972  54.9292         13.6814            0            0        41.6372      1
29298  54.9292         13.6814            0            0        41.6372      0
34163  54.9292         13.6814            0            0        41.6372      1
43881  54.9292         13.6814            0            0        41.6372      1

[45222 rows x 6 columns]
Time required for perturba

###Model

In [22]:
X_train, X_test, y_train, y_test = train_test_split(data_ready10.iloc[:,0:5], np.array(data_ready10.iloc[:,5:], dtype=int), test_size=0.2)
model10 = TrainingModel((5,))
model10.fit(X_train, y_train)
model10.evaluate(X_test, y_test, print_report=True)

Accuracy score: 0.77745
--------------------
Confusion Matrix:
[[6576  271]
 [1742  456]]
--------------------
              precision    recall  f1-score   support

           0       0.79      0.96      0.87      6847
           1       0.63      0.21      0.31      2198

    accuracy                           0.78      9045
   macro avg       0.71      0.58      0.59      9045
weighted avg       0.75      0.78      0.73      9045

--------------------
AUC score: 0.56094


##k=5

###K means

In [23]:
#將SOM資料集丟入Kmeans
#Kmeans分群數依照k值(k-anonymity的k)決定，從K = x/k開始遞減直到每一群至少有k筆資料
#計算跑的時間
#sizes = [5, 10, 15, 20, 30, 50]
k = 5
K = int(len(new_data[0])//(k*2))
D = 250//k #Decline
cluster = True
#c = Counter()

#for k in sizes:
#    print("K-Anonymity: k=" + str(k))
#    K = int(len(new_data[0])/(k*2))
#    cluster = True
kmeans_start = datetime.now()
while cluster:
    clf = MiniBatchKMeans(n_clusters=K)
    clf.fit(new_data[0])
    c = Counter(clf.labels_)
    print("K=" + str(K))
    for i in range(0,K):
        print("cluster " + str(i) + " has " + str(c[i]) + " data points")
        if c[i]<k:
            break
        else:
            if i == K-1:
                cluster = False
            else:
                pass
    K = K-D
K = K+D
print("The Resulting number of clusters: " + str(K))
print("Time required for Kmeans: " + str(datetime.now()-kmeans_start))

/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=4522. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=4522. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=4522. Setting it to 3*k
  init_size=init_size)


K=4522
cluster 0 has 2 data points


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=4472. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=4472. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=4472. Setting it to 3*k
  init_size=init_size)


K=4472
cluster 0 has 102 data points
cluster 1 has 135 data points
cluster 2 has 74 data points
cluster 3 has 229 data points
cluster 4 has 53 data points
cluster 5 has 64 data points
cluster 6 has 101 data points
cluster 7 has 30 data points
cluster 8 has 28 data points
cluster 9 has 127 data points
cluster 10 has 62 data points
cluster 11 has 69 data points
cluster 12 has 208 data points
cluster 13 has 4 data points


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=4422. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=4422. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=4422. Setting it to 3*k
  init_size=init_size)


...

K=422
cluster 0 has 45 data points
cluster 1 has 134 data points
cluster 2 has 192 data points
cluster 3 has 53 data points
cluster 4 has 87 data points
cluster 5 has 106 data points
cluster 6 has 69 data points
cluster 7 has 237 data points
cluster 8 has 86 data points
cluster 9 has 167 data points
cluster 10 has 67 data points
cluster 11 has 102 data points
cluster 12 has 160 data points
cluster 13 has 206 data points
cluster 14 has 76 data points
cluster 15 has 136 data points
cluster 16 has 76 data points
cluster 17 has 101 data points
cluster 18 has 144 data points
cluster 19 has 359 data points
cluster 20 has 84 data points
cluster 21 has 113 data points
cluster 22 has 117 data points
cluster 23 has 81 data points
cluster 24 has 8 data points
cluster 25 has 87 data points
cluster 26 has 16 data points
cluster 27 has 146 data points
cluster 28 has 62 data points
cluster 29 has 88 data points
cluster 30 has 92 data points
cluster 31 has 100 data points
cluster 32 has 181 data point

/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=372. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=372. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=372. Setting it to 3*k
  init_size=init_size)


K=372
cluster 0 has 137 data points
cluster 1 has 138 data points
cluster 2 has 139 data points
cluster 3 has 178 data points
cluster 4 has 132 data points
cluster 5 has 218 data points
cluster 6 has 190 data points
cluster 7 has 157 data points
cluster 8 has 75 data points
cluster 9 has 190 data points
cluster 10 has 194 data points
cluster 11 has 187 data points
cluster 12 has 131 data points
cluster 13 has 282 data points
cluster 14 has 78 data points
cluster 15 has 20 data points
cluster 16 has 424 data points
cluster 17 has 142 data points
cluster 18 has 29 data points
cluster 19 has 118 data points
cluster 20 has 208 data points
cluster 21 has 265 data points
cluster 22 has 52 data points
cluster 23 has 116 data points
cluster 24 has 28 data points
cluster 25 has 212 data points
cluster 26 has 201 data points
cluster 27 has 85 data points
cluster 28 has 201 data points
cluster 29 has 105 data points
cluster 30 has 52 data points
cluster 31 has 113 data points
cluster 32 has 65 da

/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=322. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=322. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=322. Setting it to 3*k
  init_size=init_size)


K=322
cluster 0 has 34 data points
cluster 1 has 149 data points
cluster 2 has 142 data points
cluster 3 has 221 data points
cluster 4 has 149 data points
cluster 5 has 158 data points
cluster 6 has 266 data points
cluster 7 has 141 data points
cluster 8 has 148 data points
cluster 9 has 287 data points
cluster 10 has 120 data points
cluster 11 has 46 data points
cluster 12 has 143 data points
cluster 13 has 195 data points
cluster 14 has 169 data points
cluster 15 has 182 data points
cluster 16 has 219 data points
cluster 17 has 115 data points
cluster 18 has 110 data points
cluster 19 has 122 data points
cluster 20 has 205 data points
cluster 21 has 148 data points
cluster 22 has 76 data points
cluster 23 has 99 data points
cluster 24 has 213 data points
cluster 25 has 208 data points
cluster 26 has 242 data points
cluster 27 has 147 data points
cluster 28 has 375 data points
cluster 29 has 226 data points
cluster 30 has 99 data points
cluster 31 has 146 data points
cluster 32 has 10

###Prepare perturbed dataset

In [24]:
#將分完群的資料還原回原始資料的feature
#準備丟入神經網路

perturb_start=datetime.now()

data = pd.concat([pd.DataFrame(new_data[1],columns=['age', 'educational-num','capital-gain', 'capital-loss', 'hours-per-week']), 
                  pd.DataFrame(new_data[2],columns=['target']), pd.DataFrame(clf.labels_,columns=['cluster'])], axis = 1)
columns = ['age', 'educational-num','capital-gain', 'capital-loss', 'hours-per-week','target','cluster']
index = range(0,len(data))
data_perturbed = pd.DataFrame(index = index, columns=columns)

for i in range(0, len(data)):
    for j in range(0,K+1):
        if data['cluster'][i] == j:
            data_perturbed[columns[0]][i] = float(data.groupby(by='cluster').mean()[columns[0]][j])
            data_perturbed[columns[1]][i] = float(data.groupby(by='cluster').mean()[columns[1]][j])
            data_perturbed[columns[2]][i] = float(data.groupby(by='cluster').mean()[columns[2]][j])
            data_perturbed[columns[3]][i] = float(data.groupby(by='cluster').mean()[columns[3]][j])
            data_perturbed[columns[4]][i] = float(data.groupby(by='cluster').mean()[columns[4]][j])
            data_perturbed['target'][i] = data['target'][i]
            data_perturbed['cluster'][i] = data['cluster'][i]

data_sorted = data_perturbed.sort_values(by=['cluster'])
data_ready5 = data_sorted.drop('cluster', axis=1)
print(data_ready5)

print("Time required for perturbation: " + str(datetime.now()-perturb_start))

           age educational-num capital-gain capital-loss hours-per-week target
10501       30               9            0            0        40.0048      0
26824       30               9            0            0        40.0048      0
33948       30               9            0            0        40.0048      0
8030        30               9            0            0        40.0048      0
13864       30               9            0            0        40.0048      0
...        ...             ...          ...          ...            ...    ...
39374  38.9775          10.073      4284.11            0        43.5899      0
34027  38.9775          10.073      4284.11            0        43.5899      0
11250  38.9775          10.073      4284.11            0        43.5899      0
42899  38.9775          10.073      4284.11            0        43.5899      1
11170  38.9775          10.073      4284.11            0        43.5899      0

[45222 rows x 6 columns]
Time required for perturba

###Model

In [25]:
X_train, X_test, y_train, y_test = train_test_split(data_ready5.iloc[:,0:5], np.array(data_ready5.iloc[:,5:], dtype=int), test_size=0.2)
model5 = TrainingModel((5,))
model5.fit(X_train, y_train)
model5.evaluate(X_test, y_test, print_report=True)

Accuracy score: 0.77247
--------------------
Confusion Matrix:
[[6510  292]
 [1766  477]]
--------------------
              precision    recall  f1-score   support

           0       0.79      0.96      0.86      6802
           1       0.62      0.21      0.32      2243

    accuracy                           0.77      9045
   macro avg       0.70      0.58      0.59      9045
weighted avg       0.75      0.77      0.73      9045

--------------------
AUC score: 0.57616
